In [85]:
import numpy as np

X = np.array([[2.5, 2.4],
              [0.5, 0.7],
              [2.2, 2.9],
              [1.9, 2.2],
              [3.1, 3.0],
              [2.3, 2.7],
              [2, 1.6],
              [1, 1.1],
              [1.5, 1.6],
              [1.1, 0.9]])

# SVD分解

$$ 
M = U \Sigma V^T \\
MM^T = U\Sigma V^T V \Sigma^T U^T \\
MM^T = U \Sigma \Sigma^T U^T \\
MM^TU = U\Sigma\Sigma^T
$$

设 $\Lambda = \Sigma\Sigma^T$, 由此可得，U矩阵是$MM^T$的特征向量，特征值是$\Lambda$

$$ 
M = U \Sigma V^T \\
M^TM = V\Sigma^TU^T U\Sigma V^T \\
M^TM = V\Sigma^T\Sigma V^T \\
M^TMV = V\Sigma^T\Sigma
$$

其中，由于$\Sigma$是对角矩阵，对称矩阵，因此$\Sigma^T\Sigma = \Sigma\Sigma^T = \Lambda$

由此可得，V矩阵是$M^TM$的特征向量，特征值是$\Lambda$

In [86]:
def svd(M: np.ndarray):
    """
    手动实现 SVD 分解
    - U 是 M @ M^T 的特征向量
    - V 是 M^T @ M 的特征向量
    - S 是奇异值，按照降序排列
    """
    # 计算 M @ M^T 和 M^T @ M 的特征值和特征向量
    MMT = M @ M.T
    MTM = M.T @ M

    # 计算特征值和特征向量
    LAMBDA_U, U = np.linalg.eigh(MMT) #np.linalg.eigh得到的特征值不是SVD需要是降序排列
    LAMBDA_V, V = np.linalg.eigh(MTM)

    # 对奇异值进行降序排列
    sorted_indices = np.argsort(-LAMBDA_U)  # 按降序排列
    U = U[:, sorted_indices]

    # V 需要按照相同的顺序调整
    sorted_indices_V = np.argsort(-LAMBDA_V)
    V = V[:, sorted_indices_V]

    # 计算奇异值
    if len(LAMBDA_V) < len(LAMBDA_U):
        S = np.sqrt(LAMBDA_V[sorted_indices_V])
    else:
        S = np.sqrt(LAMBDA_U[sorted_indices])

    # 修正符号：调整 `U` 和 `V` 的列符号，使结果一致
    for i in range(len(S)):
        if np.sign(U[0, i]) != np.sign(V[0, i]):
            V[:, i] *= -1

    return U, S, V.T


In [87]:
# 计算 SVD
U, S, Vt = svd(X)
print(U.shape, S.shape, Vt.shape)
# 输出结果
print("U 矩阵：")
print(U[:2, :2])
print("\n奇异值：")
print(S)
print("\nV 转置矩阵：")
print(Vt)

(10, 10) (2,) (2, 2)
U 矩阵：
[[-0.38507927  0.25575629]
 [-0.09481302 -0.17561812]]

奇异值：
[8.98868529 0.66598554]

V 转置矩阵：
[[-0.68647784 -0.72715072]
 [ 0.72715072 -0.68647784]]


In [88]:
# 可以复现原来的值
U[:,:2] @ np.diag(S) @ Vt

array([[2.5, 2.4],
       [0.5, 0.7],
       [2.2, 2.9],
       [1.9, 2.2],
       [3.1, 3. ],
       [2.3, 2.7],
       [2. , 1.6],
       [1. , 1.1],
       [1.5, 1.6],
       [1.1, 0.9]])

### 对比调包的结果

对比发现，结果一致

In [89]:
# 计算 SVD
U, S, Vt = np.linalg.svd(X)
print(U.shape, S.shape, Vt.shape)
# 输出结果
print("U 矩阵：")
print(U[:2, :2])
print("\n奇异值：")
print(S)
print("\nV 转置矩阵：")
print(Vt)

(10, 10) (2,) (2, 2)
U 矩阵：
[[-0.38507927  0.25575629]
 [-0.09481302 -0.17561812]]

奇异值：
[8.98868529 0.66598554]

V 转置矩阵：
[[-0.68647784 -0.72715072]
 [ 0.72715072 -0.68647784]]


# PCA

PCA的过程主要分两步
1. 去中心化
2. 通过协方差矩阵计算R矩阵

关于第二个步骤：

说明：$X_i$是一个行向量，确保数据矩阵的每一列是一个特征，每一行是一个样本。

$cov(X) = \frac{1}{n-1} X^TX$

R矩阵就是cov(X)的特征向量，解释方差就是cov(X)的特征值

由上面已知：V是$X^TX$的特征向量，$\Lambda$是$X^TX$的特征值。

所以，R就是SVD中的V，解释方差就是$\frac{\Sigma^T \Sigma}{n-1}$,其中n是样本数量。

In [90]:
def pca(X, n_components):

    # 去中心化
    X_mean = np.mean(X, axis=0)
    X_centered = X - X_mean

    # 计算旋转矩阵
    U, S, Vt = svd(X_centered)

    # 确保符号一致，调整 U 和 Vt 的符号
    for i in range(n_components):
        if np.sum(U[:, i]) < 0:
            U[:, i] = -U[:, i]
            Vt[i, :] = -Vt[i, :]

    # 旋转矩阵R就是V
    R = Vt.T[:, :n_components]

    # 计算解释方差
    explained_variance = S**2 / (len(X) - 1)

    # 数据投影到新空间
    X_pca = X_centered @ R

    return X_pca, R, explained_variance

    

In [91]:
# 执行 PCA
n_components = 2  # 降维到 2 个主成分
X_pca, components, explained_variance = pca(X, n_components)

# 输出结果
print("降维后的数据：")
print(X_pca)
print("\n主成分方向：")
print(components)
print("\n解释方差：")
print(explained_variance)

降维后的数据：
[[ 0.82797019 -0.17511531]
 [-1.77758033  0.14285723]
 [ 0.99219749  0.38437499]
 [ 0.27421042  0.13041721]
 [ 1.67580142 -0.20949846]
 [ 0.9129491   0.17528244]
 [-0.09910944 -0.3498247 ]
 [-1.14457216  0.04641726]
 [-0.43804614  0.01776463]
 [-1.22382056 -0.16267529]]

主成分方向：
[[ 0.6778734  -0.73517866]
 [ 0.73517866  0.6778734 ]]

解释方差：
[1.28402771 0.0490834 ]


### 对比调包的结果

对比发现，结果一致

In [92]:
from sklearn.decomposition import PCA

# 创建 PCA 模型，指定目标维度
n_components = 2
pca = PCA(n_components=n_components)

# 拟合数据并进行降维
X_pca = pca.fit_transform(X)

# 输出结果
print("降维后的数据：")
print(X_pca)
print("\n主成分方向：")
print(pca.components_)  # 每一行是一个主成分方向向量
print("\n解释方差：")
print(pca.explained_variance_)


降维后的数据：
[[ 0.82797019  0.17511531]
 [-1.77758033 -0.14285723]
 [ 0.99219749 -0.38437499]
 [ 0.27421042 -0.13041721]
 [ 1.67580142  0.20949846]
 [ 0.9129491  -0.17528244]
 [-0.09910944  0.3498247 ]
 [-1.14457216 -0.04641726]
 [-0.43804614 -0.01776463]
 [-1.22382056  0.16267529]]

主成分方向：
[[ 0.6778734   0.73517866]
 [ 0.73517866 -0.6778734 ]]

解释方差：
[1.28402771 0.0490834 ]
